In [1]:
import os
import json
from pandas.io.json import json_normalize
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

import warnings
# current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

import gc
import datetime

from sklearn.preprocessing import LabelEncoder

%matplotlib inline
color = sns.color_palette()

Loading data

In [2]:
df_train = pd.read_csv('input/train-flattened.csv',
                       dtype={'fullVisitorId': np.str})  # MUST change the fullVisitorId to string format (REQUIRED!)
df_test = pd.read_csv('input/test-flattened.csv',
                       dtype={'fullVisitorId': np.str})

In [3]:
# replace NaN with zero for column - totals.transactionRevenue  (only do this for training data)
df_train['totals.transactionRevenue'].fillna(0, inplace=True)

const_cols = [c for c in df_train.columns if df_train[c].nunique(dropna=False)==1 ]

df_train_clean = df_train.drop(const_cols, axis=1)
df_test_clean = df_test.drop(const_cols, axis=1)

# Drop useless features
df_train_clean.drop(['sessionId', "trafficSource.campaignCode"], axis=1, inplace=True)
df_test_clean.drop(['sessionId'], axis=1, inplace=True)


Preprocessing for categorical features

In [4]:
# specify which categorical variables to replace NaN (and label encoding for later)
cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']

for col in cat_cols:    
    # Replace NaN with 'missing'
    df_train_clean[col] = df_train_clean[col].fillna('missing')
    df_test_clean[col] = df_test_clean[col].fillna('missing')

In [5]:
def clean_small_cap(df):
    for col in cat_cols:
        #print('Different elements in the feature:', df_train_clean[col].unique())
        before = df[col].unique()
        cleaned = np.unique([str(x).lower() for x in df[col]])

        if len(before) == len(cleaned):
            # There is no small/capital letter issue:
            print("no issue with {}".format(col))
        else:
            print("THERE ISSSSS with {}".format(col))
            df[col] = [x.lower() for x in df[col]]

        

In [6]:
clean_small_cap(df_train_clean)

no issue with channelGrouping
no issue with device.browser
no issue with device.deviceCategory
no issue with device.operatingSystem
no issue with geoNetwork.city
no issue with geoNetwork.continent
no issue with geoNetwork.country
no issue with geoNetwork.metro
no issue with geoNetwork.networkDomain
no issue with geoNetwork.region
no issue with geoNetwork.subContinent
THERE ISSSSS with trafficSource.adContent
no issue with trafficSource.adwordsClickInfo.adNetworkType
no issue with trafficSource.adwordsClickInfo.gclId
no issue with trafficSource.adwordsClickInfo.page
no issue with trafficSource.adwordsClickInfo.slot
no issue with trafficSource.campaign
THERE ISSSSS with trafficSource.keyword
no issue with trafficSource.medium
THERE ISSSSS with trafficSource.referralPath
no issue with trafficSource.source
no issue with trafficSource.adwordsClickInfo.isVideoAd
no issue with trafficSource.isTrueDirect


In [7]:
clean_small_cap(df_test_clean)

no issue with channelGrouping
no issue with device.browser
no issue with device.deviceCategory
no issue with device.operatingSystem
no issue with geoNetwork.city
no issue with geoNetwork.continent
no issue with geoNetwork.country
no issue with geoNetwork.metro
no issue with geoNetwork.networkDomain
no issue with geoNetwork.region
no issue with geoNetwork.subContinent
THERE ISSSSS with trafficSource.adContent
no issue with trafficSource.adwordsClickInfo.adNetworkType
no issue with trafficSource.adwordsClickInfo.gclId
no issue with trafficSource.adwordsClickInfo.page
no issue with trafficSource.adwordsClickInfo.slot
no issue with trafficSource.campaign
THERE ISSSSS with trafficSource.keyword
no issue with trafficSource.medium
THERE ISSSSS with trafficSource.referralPath
no issue with trafficSource.source
no issue with trafficSource.adwordsClickInfo.isVideoAd
no issue with trafficSource.isTrueDirect


In [8]:
df_train_clean.corr()

,date,visitId,visitNumber,visitStartTime,device.isMobile,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue
date,1.000000,0.880936,0.007536,0.880936,0.146148,NaN,-0.018395,NaN,-0.012146,0.003188
visitId,0.880936,1.000000,0.002069,1.000000,0.144332,NaN,-0.027701,NaN,-0.022259,0.002724
visitNumber,0.007536,0.002069,1.000000,0.002069,-0.037667,NaN,0.041317,NaN,0.043363,0.051366
visitStartTime,0.880936,1.000000,0.002069,1.000000,0.144332,NaN,-0.027701,NaN,-0.022259,0.002724
device.isMobile,0.146148,0.144332,-0.037667,0.144332,1.000000,NaN,-0.030367,NaN,-0.030199,-0.016555
totals.bounces,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
totals.hits,-0.018395,-0.027701,0.041317,-0.027701,-0.030367,NaN,1.000000,NaN,0.983205,0.154333
totals.newVisits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
totals.pageviews,-0.012146,-0.022259,0.043363,-0.022259,-0.030199,NaN,0.983205,NaN,1.000000,0.155590
totals.transactionRevenue,0.003188,0.002724,0.051366,0.002724,-0.016555,NaN,0.154333,NaN,0.155590,1.000000


Preprocessing for numerical features

In [10]:
# specify which numerical variables to replace NaN
num_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    
    # convert numerical variables to float
    # Replace NaN with 0
    df_train_clean[col] = df_train_clean[col].astype('float').fillna(0)
    df_test_clean[col] = df_test_clean[col].astype('float').fillna(0)

Preprocessing for other type of features (Boolean value, dates)

In [9]:
df_train_clean['device.isMobile'] = df_train_clean['device.isMobile'].astype(int)
df_test_clean['device.isMobile'] = df_test_clean['device.isMobile'].astype(int)

# change int format to string format for the column - date
df_train_clean['date'] = df_train_clean['date'].astype(str)
df_test_clean['date'] = df_test_clean['date'].astype(str)

# add a new column - yearmonth
df_train_clean.insert(loc=2, column='yearmonth', value=df_train_clean['date'].str.slice(start=0, stop=-2))
df_test_clean.insert(loc=2, column='yearmonth', value=df_test_clean['date'].str.slice(start=0, stop=-2))

df_train_clean['date'] = pd.to_datetime(df_train_clean['date'], format='%Y%m%d')
df_test_clean['date'] = pd.to_datetime(df_test_clean['date'], format='%Y%m%d')

df_train_clean = df_train_clean.sort_values(by='date', ascending=True)
df_test_clean = df_test_clean.sort_values(by='date', ascending=True)



In [11]:
df_train_clean = df_train_clean.sort_values(by='date', ascending=True)
df_test_clean = df_test_clean.sort_values(by='date', ascending=True)

# training data
df_train_clean.insert(loc=2, column='year', value=df_train_clean.date.dt.year)
df_train_clean.insert(loc=3, column='month', value=df_train_clean.date.dt.month)
# +1 to make Monday=1.....until Sunday=7
df_train_clean.insert(loc=4, column='day', value=(df_train_clean.date.dt.dayofweek)+1)

# testing data
df_test_clean.insert(loc=2, column='year', value=df_test_clean.date.dt.year)
df_test_clean.insert(loc=3, column='month', value=df_test_clean.date.dt.month)
# +1 to make Monday=1.....until Sunday=7
df_test_clean.insert(loc=4, column='day', value=(df_test_clean.date.dt.dayofweek)+1)



Label Encoding

In [12]:
# reset index after we rearranged the rows based on date
df_train_clean.reset_index(drop=True, inplace=True)
df_test_clean.reset_index(drop=True, inplace=True)

df_train_clean["totals.transactionRevenue"] = df_train_clean["totals.transactionRevenue"].astype('float')

# Loop through each categorical column
for col in cat_cols:
    label_encoder = LabelEncoder()
    # use the label encoding based on training and testing data to capture all strings
    label_encoder.fit(list(df_train_clean[col].values.astype('str')) + list(df_test_clean[col].values.astype('str')))
    df_train_clean[col] = label_encoder.transform(list(df_train_clean[col].values.astype('str')))
    df_test_clean[col] = label_encoder.transform(list(df_test_clean[col].values.astype('str')))
    print('Label encoded: {}'.format(col))

Label encoded: channelGrouping
Label encoded: device.browser
Label encoded: device.deviceCategory
Label encoded: device.operatingSystem
Label encoded: geoNetwork.city
Label encoded: geoNetwork.continent
Label encoded: geoNetwork.country
Label encoded: geoNetwork.metro
Label encoded: geoNetwork.networkDomain
Label encoded: geoNetwork.region
Label encoded: geoNetwork.subContinent
Label encoded: trafficSource.adContent
Label encoded: trafficSource.adwordsClickInfo.adNetworkType
Label encoded: trafficSource.adwordsClickInfo.gclId
Label encoded: trafficSource.adwordsClickInfo.page
Label encoded: trafficSource.adwordsClickInfo.slot
Label encoded: trafficSource.campaign
Label encoded: trafficSource.keyword
Label encoded: trafficSource.medium
Label encoded: trafficSource.referralPath
Label encoded: trafficSource.source
Label encoded: trafficSource.adwordsClickInfo.isVideoAd
Label encoded: trafficSource.isTrueDirect


## Data preparation

In [13]:
unused_feature = [
'totals.transactionRevenue',
 'yearmonth',
 'date',
 'totals.bounces',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'trafficSource.adwordsClickInfo.slot',
 'trafficSource.adwordsClickInfo.page',
 'trafficSource.adwordsClickInfo.isVideoAd',
 'trafficSource.campaign',
 'trafficSource.adContent',
 'device.deviceCategory',
 'geoNetwork.subContinent',
 'year'
]

In [14]:
train_data = df_train_clean[df_train_clean.columns[~df_train_clean.columns.isin(unused_feature)]]

In [15]:
train_data.head()

,channelGrouping,month,day,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,device.isMobile,device.operatingSystem,...,geoNetwork.region,totals.hits,totals.newVisits,totals.pageviews,trafficSource.adwordsClickInfo.gclId,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,2,8,1,1492602573213666603,1470044332,1.0,1.470044e+09,35,0,7,...,482,1.0,1.0,1.0,59008,0,2742,0,3192,0
1,4,8,1,4703900907627844850,1470066190,2.0,1.470066e+09,35,0,20,...,338,1.0,0.0,1.0,59008,1,9,5,3192,208
2,2,8,1,5408757094730735604,1470039998,1.0,1.470040e+09,35,0,20,...,482,1.0,1.0,1.0,59008,0,2742,0,3192,0
3,2,8,1,893148638848434176,1470118400,1.0,1.470118e+09,35,0,20,...,482,1.0,1.0,1.0,59008,0,2742,0,3192,0
4,2,8,1,6449391852916941365,1470075217,1.0,1.470075e+09,72,0,7,...,192,1.0,1.0,1.0,59008,0,2742,0,3192,0


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

# Helper function
def train_valid_split_wrt_time(data, feature_column, percent_valid = 0.1):
    last_indi_train = int(np.floor(data.shape[0]*(1-percent_valid)))
    x_train, x_valid = data[feature_column].iloc[0:last_indi_train], \
                    data[feature_column].iloc[last_indi_train:]
        
    y_train, y_valid = data['totals.transactionRevenue'].iloc[0:last_indi_train], \
                    data['totals.transactionRevenue'].iloc[last_indi_train:]
        
    return x_train, x_valid, y_train, y_valid 

def timeseries_cv(X, Y, model, cv=5):
    mse=[]
    tscv = TimeSeriesSplit(n_splits=cv)
    for train_index, test_index in tscv.split(X): 
        X_train, X_test = X.iloc[train_index] , X.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index] , Y.iloc[test_index]
        clf4_rf = model
        clf4_rf.fit(X_train,Y_train)
        
        X_test['totals.transactionRevenue_predicted'] = clf4_rf.predict(X_test)
        
        X_test_pred = X_test[['fullVisitorId','totals.transactionRevenue_predicted']]
        X_test_pred['totals.transactionRevenue'] = Y_test
        
        X_test_pred = X_test_pred.groupby('fullVisitorId').agg({'totals.transactionRevenue_predicted':'sum',
                                                               'totals.transactionRevenue':'sum'}).reset_index()
    
        
        error = np.sqrt(mean_squared_error(X_test_pred['totals.transactionRevenue'].apply(lambda x: np.log1p(x)),
                                           X_test_pred['totals.transactionRevenue_predicted'].apply(lambda x: np.log1p(x))))
        print('Error: {}'.format(error))
        mse.append(error)
    
    return np.mean(mse)
        

===TESTING===

## Try LGBM
#### For installation of lightgbm 
#### https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html#macos
#### https://github.com/Microsoft/LightGBM/issues/1456

In [ ]:
df_train_clean.columns

In [ ]:
df_train_clean_grp = df_train_clean.groupby(
    ["fullVisitorId"]
    ).agg({'totals.transactionRevenue':'sum', 'visitNumber':'sum', 'totals.pageviews':'sum', 
           'totals.hits':'sum'})

In [16]:
df_train_clean_grp.head()

NameError: name 'df_train_clean_grp' is not defined

In [ ]:
df_train_clean_grp['totals.transactionRevenue'] = np.log1p(df_train_clean_grp['totals.transactionRevenue'])

In [ ]:
df_train_clean_grp.head()

In [ ]:
df_test_clean_grp = df_test_clean.groupby(
    ["fullVisitorId"]
    ).agg({'visitNumber':'sum', 'totals.pageviews':'sum', 
           'totals.hits':'sum', ''})

In [ ]:
df_test_clean_grp.head()

In [ ]:
# Split the train dataset into development and valid based on time
feature_column_1 = ['visitNumber', 'totals.pageviews','totals.hits']
x_train, x_test, y_train, y_test = train_valid_split_wrt_time(df_train_clean_grp, feature_column_1)


# custom function to run light gbm model
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 30,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    return pred_test_y, model, pred_val_y

# Training the model #
pred_test, model, pred_val = run_lgb(x_train, y_train, x_test, y_test, df_test_clean_grp)

In [ ]:
df_test_clean_grp.shape

In [ ]:
len(pred_test)

In [ ]:
df_test_clean_grp['PredictedLogRevenue'] = pred_test

In [ ]:
df_test_clean_grp.shape

In [ ]:
df_test_clean_grp[['PredictedLogRevenue']].to_csv('out_lgbm.csv')

===End of TESTING==

Recover geoNetwork source

In [ ]:
df_train_clean['geoNetwork.continent'].unique()

In [ ]:
df_train_clean['geoNetwork.subContinent'].unique()[3]

In [ ]:
df_train_clean['geoNetwork.region'].unique()[3]

In [ ]:
df_train_clean['geoNetwork.country'].unique()[0]

In [ ]:
df_train_clean['geoNetwork.city'].unique()[3]

In [ ]:
df_train_clean['geoNetwork.metro'].unique()[4]

In [ ]:
df_train_clean[df_train_clean['geoNetwork.metro'] == 'Houston TX']['geoNetwork.city'].unique()

In [ ]:
df_train_clean['geoNetwork.networkDomain'].unique()[0]

In [ ]:
df_train_clean.columns